In [32]:
import json
import ollama
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [71]:
start_date = '2022-09-30' # start_date
end_date = '2023-09-30' # end_date
id_object = '少爷和我' # id对象选择
language = 'en' # 获取的评论语言
d_num = 100 # 需要使用的评论数
f_num = 2 # 获取的文件范围
id_type = 'IP' # id对象类型-用来筛选Aspect词典

from data_pro import data_select
from youtube_ip_Language_recognition_s3_v1 import remove_emoji, remove_punctuations, remove_numbers, regularize
data_df = data_select(start_date, end_date, id_object, language, d_num, f_num) # 只用了id_name, title, 和comment_content
data_df['comment_content'] = (data_df['comment_content']
    .apply(remove_emoji)
    .apply(remove_punctuations)
    .apply(remove_numbers)
    .apply(regularize)
)

/Users/qiaochufeng/Documents/GitHub/Yoywow-Internship/Code/8.NER_ALL/data_pro.py:17: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'../../Data/Sentiment/Test_data/part-0000{i}-3d63779d-fee1-4706-a8e5-db2989c81cb5-c000.csv', on_bad_lines='warn')


In [34]:
def llama3_chat(prompt, ground_knowledge=""):
    
    messages = []
    if ground_knowledge:
        messages.append({"role": "system", "content": ground_knowledge})
    messages.append({"role": "user", "content": prompt})
    response = ollama.chat(model='llama3', messages=messages)

    return response['message']['content']

def groundknowledge(knowdict, keyword):
    try:
        knowitem = knowdict[keyword]
        return knowitem
    except:
        return ""

def generate_prompt(prompt_template, id_object, type_schema, format, text):
    instruct = prompt_template.format(id_object=id_object, type_schema=type_schema, format=format, text=text)
    return instruct

In [81]:
prompt_template = (
            "'instruction':'You are an expert in named entity recognition. "
                "task: NER"
                "The input text is a web comment about {id_object}, please try to understand it with your knowledge."
                "Please extract the main aspect of comment that match from:{type_schema}"
                "Generate the main entity of comment, the entity should not be the name of {id_object}, it should be a specific thing related to the aspect."
                "Then generate 3 topic words that can conclude the content about the entity."
                "Please return an empty output if the comment has not relation with {id_object}"
                "Please respond in the json format {format}.', \n"
            "'input':'{text}' \n"
            "'result':")

format = """
{"aspect": "",
"entity": "",
"topic": ""}
"""

knowdict = {"王鹤棣": "王鹤棣 (Wang Hedi) is a young Chinese actor and model who has gained popularity in recent years. Here are some key points about him: Birth: He was born on December 20, 1997, in Shenyang, Liaoning Province, China. Career start: Wang began his career as a model and later transitioned into acting. Breakthrough role: He gained significant attention for his role as Gu Weiyi in the 2022 drama ‘Love Between Fairy and Devil’ (苍兰诀), which became a major hit. Other notable works: He has appeared in several other TV dramas, including ‘The Whirlwind Girl 2’ (旋风少女2) and ‘Stand By Me’ (一起同过窗). Popularity: Wang has become quite popular among younger audiences in China, particularly for his good looks and acting skills. Modeling: He continues to work as a model and has been associated with various fashion brands. Social media presence: Wang is active on social media platforms like Weibo, where he has a large following.",
            "少爷和我": "少爷和我 (Our Parallel Universe) is a Chinese television drama that combines elements of romance and comedy. The series revolves around the intriguing dynamics between a wealthy young master and his new personal assistant. Set against a backdrop of luxury and high society, the show explores themes of love, power, and personal growth as the characters navigate their complex relationships and the challenges that come with their distinct social standings. This engaging series captivates viewers with its witty dialogue, compelling plot, and heartfelt moments.",
            "Gen1es": "Gen1es is a multi-national girl group based in Thailand under Ryce Entertainment. They were formed through the 2024 reality show Chuang Asia: Thailand on WeTV. The group consists of nine members: Pailiu, Wang Ke, Xueyao, Qiao Yiyu, Elyn, Yean, Ruan, Didi and Emma. They are set to debut in 2024,[2] and will be active for three years.[3]"}

Comments = data_df['comment_content'].to_list()[:30]

In [79]:
with open('Entity_list.json', 'r', encoding='utf-8') as json_file:
    entity_dict = json.load(json_file)

def tag_com(prompt_template, id_object, type_schema, format, Comments):
    
    results = []

    for text in Comments:

        output = llama3_chat(
            prompt = generate_prompt(prompt_template=prompt_template, id_object=id_object, type_schema=type_schema, format=format, text=text), 
            ground_knowledge = groundknowledge(knowdict=knowdict, keyword=id_object)
        )

        str_data = (output[output.find("{"): output.find("}")+1])
        json_data = json.loads(str_data)
        json_data['comment'] = text

        results.append(json_data)
        with open(f'{id_object}_output.json', 'w', encoding='utf-8') as json_file:
            json.dump(results, json_file, ensure_ascii=False, indent=4)

        print(json_data)

    return results

In [69]:
# 王鹤棣
results = tag_com(prompt_template=prompt_template, id_object = id_object, type_schema=entity_dict[id_type], format=format, Comments=Comments)
results

{'entity_type': '', 'entity': '', 'topic': [], 'comment': 'ok  yes her schedule shows she s flying to BJ on the th And today is the th in china  I m working on the video'}
{'entity_type': 'Personality', 'entity': 'good looks', 'topic': ['looks', 'popular', 'actor'], 'comment': 'where is kun'}
{'entity_type': 'Relationships', 'entity': 'Dyshen couple', 'topic': ['cute', 'funny', 'love'], 'comment': ' Rosa Bombai Yes my dear friendi agree with you in everything you write in comments  Shen Yue is very cute and funny like Dylan they have the same characters  I believe that s why Dylan love Shen Yue so much because she has natural beauty and beautiful character  When the Dyshen couple they are together they can t hide their affection and their feelings  And when we see them together we can feel their electricity and their love  I believe that the Dyshen couple they know how to move and maybe they can be togetherin some places in private  Yesterday Dylan wearing a t shirt with the phrase  so

[{'entity_type': '',
  'entity': '',
  'topic': [],
  'comment': 'ok  yes her schedule shows she s flying to BJ on the th And today is the th in china  I m working on the video'},
 {'entity_type': 'Personality',
  'entity': 'good looks',
  'topic': ['looks', 'popular', 'actor'],
  'comment': 'where is kun'},
 {'entity_type': 'Relationships',
  'entity': 'Dyshen couple',
  'topic': ['cute', 'funny', 'love'],
  'comment': ' Rosa Bombai Yes my dear friendi agree with you in everything you write in comments  Shen Yue is very cute and funny like Dylan they have the same characters  I believe that s why Dylan love Shen Yue so much because she has natural beauty and beautiful character  When the Dyshen couple they are together they can t hide their affection and their feelings  And when we see them together we can feel their electricity and their love  I believe that the Dyshen couple they know how to move and maybe they can be togetherin some places in private  Yesterday Dylan wearing a t sh

In [80]:
# 少爷和我
results = tag_com(prompt_template=prompt_template, id_object = id_object, type_schema=entity_dict[id_type], format=format, Comments=Comments)
results

{'aspect': 'Plot', 'entity': 'Parallel Universe', 'topic': ['Reality', 'Identity', 'Connection'], 'comment': 'Do you know much about disassociation   is it a similar thing happening    connecting to parallel version of yourself  '}
{'aspect': 'Audience Reflections', 'entity': 'Plot', 'topic': ['Drama', 'Romance', 'Comedy'], 'comment': 'https   youtu be yArprkqeE this one'}
{'aspect': '', 'entity': 'Consciousness', 'topic': ['vibration', 'love', 'contagious'], 'comment': 'When you re speaking about perceiving someone you meet as being in a higher vibration because you re in a contagious higher vibration you remind me of what my teacher Ivan Rados says   Consciousness is Contagious  Love it'}
{'aspect': 'Audience Reflections', 'entity': 'Sibling Relationships', 'topic': ['Family', 'Age Gap', 'Bonding'], 'comment': 'Speaking from my experience my sister and I are a year apart  me older  and weren t friends until we were mid teens  But my brother was is  years older than me and we were clo

[{'aspect': 'Plot',
  'entity': 'Parallel Universe',
  'topic': ['Reality', 'Identity', 'Connection'],
  'comment': 'Do you know much about disassociation   is it a similar thing happening    connecting to parallel version of yourself  '},
 {'aspect': 'Audience Reflections',
  'entity': 'Plot',
  'topic': ['Drama', 'Romance', 'Comedy'],
  'comment': 'https   youtu be yArprkqeE this one'},
 {'aspect': '',
  'entity': 'Consciousness',
  'topic': ['vibration', 'love', 'contagious'],
  'comment': 'When you re speaking about perceiving someone you meet as being in a higher vibration because you re in a contagious higher vibration you remind me of what my teacher Ivan Rados says   Consciousness is Contagious  Love it'},
 {'aspect': 'Audience Reflections',
  'entity': 'Sibling Relationships',
  'topic': ['Family', 'Age Gap', 'Bonding'],
  'comment': 'Speaking from my experience my sister and I are a year apart  me older  and weren t friends until we were mid teens  But my brother was is  year

In [ ]:
# Gen1es
results = tag_com(prompt_template=prompt_template, id_object = id_object, type_schema=entity_dict[id_type], format=format, Comments=Comments)
results